In [1]:
def subdivizija(G):
    # naredi subdivizijo grafa, na vsako povezavo postavi eno vozlišče
    H = G.copy()
    H.subdivide_edges(G.edges(), 1)
    return H

def generiraj_subkubične_grafe(n):
    # generira vse subkubične grafe na n vozliščih
    return [subdivizija(G) for G in graphs.nauty_geng(f"{n} -c -D3")]

# slovar barv, ki jih bomo uporabili za risanje grafov
barve = {1 : "blue", 2 : "green", 3 : "red", 4 : "orange", 5 : "purple", 6 : "gold", 7 : "gray"}

def prikazi_graf(G, pakirno_stevilo, barve_vozlisc):
    # izpiše pakirno število, slovar barv vozlišč in nariše graf
    
    # ustvari slovar barv s katerim bomo pobarvali vozlišča
    slovar_barv = {}
    for v, b in barve_vozlisc.items():
        if barve[b] in slovar_barv.keys():
            slovar_barv[barve[b]].append(v)
        else:
            slovar_barv[barve[b]] = [v]

    plt = G.plot(vertex_color=slovar_barv)
    plt.show()
    print(barve_vozlisc, f"Pakirno kromatično število: {pakirno_stevilo}")

def nalozi_seznam(ime_dat):
    # naloži seznam shranjen v datoteki
    # ime_dat je treba podati kot string
    return load(ime_dat)

def prikazi_rezultate(ime_dat):
    # prikaže grafe shranjene kot seznam v datoteki ime_dat
    # ime_dat je treba podati kot string
    
    for el in nalozi_seznam(ime_dat):
        G, pakirno_stevilo, barve_vozlisc = el
        prikazi_rezultat(G, pakirno_stevilo, barve_vozlisc)
        
    print("KONČANO")

In [2]:
def poisci_pakirno_stevilo(G, max_colors, min_packing_number):
    # poišče pakirno število grafa G, če je le to >= min_packing_number. Št. barv <= max_colors
    # vrne trojico: graf G, pakirno število grafa, slovar vozlišč s pripadajočimi barvami
    
    V = G.vertices()
    D = G.distance_matrix()
    
    # ustvarimo CLP
    p = MixedIntegerLinearProgram(maximization=False)
    x = p.new_variable(binary=True) # x_v,i -> vozlišče v je pobarvano z barvo i
    z = p.new_variable(nonnegative=True) # šteje koliko barv smo porabili (pakirno število)
    
    # minimiziramo število uporabljenih barv
    p.set_objective(z[0])
    
    # omejitve:
    # 1) vsako vozlišče je pobarvano z natanko eno barvo
    for v in V:
        p.add_constraint(sum(x[v, i] for i in range(1, max_colors + 1)) == 1)
    
    # 2) če sta vozlišči oddaljeni <i+1 sta pobarvani z različno barvo
    for v in V:
        for u in V:
            for i in range(1, max_colors + 1): # range(1, D[v][u] + 1)
                if (D[v][u] <= i) and (v != u):
                    p.add_constraint(x[v, i] + x[u, i] <= 1)
    
    # 3) barva vsakega vozlišča ni večja od pakirnega števila
    for v in V:
        for i in range(1, max_colors + 1):
            p.add_constraint(i * x[v, i] <= z[0])                

    # reši CLP
    p.solve()
    [x_sol, z_sol] = p.get_values(x, z[0]) 

    # vrnemo le grafe s pakirnim številom >= min_packing_number
    if z_sol >= min_packing_number:
        # ustvari slovar, ključi so vozlišča, vrednosti so barve vozlišč
        barve_vozlisc = {v: next(i for i in range(1, max_colors + 1) if x_sol[v, i] > 0.5) for v in V}
        return (G, z_sol, barve_vozlisc)
    else:
        return

In [ ]:
# ne shrani grafov v seznam

#def izvedi_iskanje(n, max_colors, min_packing_number):
    # požene CLP na vseh subdivizijah grafov na n vozliščih in vrne seznam tistih s pakirnim številom >= min_packing_number

#    for G in generiraj_subkubične_grafe(n):
#        rezultat = poisci_pakirno_stevilo(subdivizija(G), max_colors, min_packing_number)
#        if rezultat is not None:
#            G, pakirno_stevilo, barve_vozlisc = rezultat
#            prikazi_graf(G, pakirno_stevilo, barve_vozlisc)
#    print("KONČANO")

#izvedi_iskanje(5, 5, 4)

In [10]:
def izvedi_iskanje(n, max_colors, min_packing_number, ime_dat):
    # požene CLP na vseh subdivizijah grafov na n vozliščih, prikaže tiste s pakirnim številom >= min_packing_number
    # in jih shrani kot seznam v datoteko ime_dat.
    # ime_dat je treba podati kot string

    sez_grafov = []
    for G in generiraj_subkubične_grafe(n):
        rezultat = poisci_pakirno_stevilo(G, max_colors, min_packing_number)
        if rezultat is not None:
            sez_grafov.append(rezultat)
            G, pakirno_stevilo, barve_vozlisc = rezultat
            prikazi_graf(G, pakirno_stevilo, barve_vozlisc)

    # shrani seznam rezultatov
    save(sez_grafov, ime_dat)

    # neki graf 6 da ti shrani graf u csv
    
    print("KONČANO")